In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import torch
from transformers import BertConfig
from fastformer import FastformerEncoder

In [ ]:
config=BertConfig.from_json_file('fastformer.json')

In [ ]:
model = FastformerEncoder(config)

In [ ]:
batch_size = 8
seq_len = 50 
dummy_input = torch.randn(batch_size, seq_len, config.hidden_size)
dummy_mask = torch.ones(batch_size, seq_len)

In [ ]:
model.eval()
with torch.no_grad():
    output = model(dummy_input, dummy_mask)

In [ ]:
print(dummy_input.shape)
print(output.shape)

In [1]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = BertModel.from_pretrained('bert-base-uncased')

c:\Users\madmo\miniconda3\envs\dl\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = bert(**encoded_input)
print(output.last_hidden_state[0].shape)

torch.Size([12, 768])


In [ ]:
print(f"Token: {tokenizer.pad_token} | ID: {tokenizer.pad_token_id}")

In [55]:
from data_utils import build_user_id_map, build_news_id_map, build_news_token, build_edge_index, build_behaviors
import pickle 

# build_news_id_map()
# build_user_id_map()
# build_news_token(news_id_map)

In [35]:
import numpy as np
news_token = np.load('data/MINDsmall_train/news_token.npy')

In [ ]:
news_token = torch.from_numpy(news_token)
attention_mask = (news_token != 0).long()

In [36]:
with open('data/MINDsmall_train/user_id_map.pkl', 'rb') as f:
    user_id_map = pickle.load(f)

In [44]:
with open('data/MINDsmall_train/news_id_map.pkl', 'rb') as f:
    news_id_map = pickle.load(f)

In [56]:
data_path = 'data/MINDsmall_train/'
edge_index = build_edge_index(data_path + 'behaviors.tsv', news_id_map=news_id_map)

100%|██████████| 153727/153727 [00:03<00:00, 47888.50it/s]


In [57]:
build_behaviors(data_path + 'behaviors.tsv', user_id_map, news_id_map, 'behaviors.pkl', 50)

156965it [00:03, 51401.92it/s]


In [69]:
with open('behaviors.pkl', 'rb') as f:
    behaviors_data = pickle.load(f)

In [3]:
from data import MINDDataset
from data_utils import collate_fn

c:\Users\madmo\miniconda3\envs\dl\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
mind_data = MINDDataset('behaviors.pkl')

In [5]:
from torch.utils.data import DataLoader

dataloader = DataLoader(mind_data, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [6]:
batch = next(iter(dataloader))

In [8]:
print(batch['user_idx'].shape)
print(batch['history'].shape)
print(batch['candidates'].shape)
print(batch['seed_nodes'].shape)

torch.Size([32])
torch.Size([32, 50])
torch.Size([32, 5])
torch.Size([784])
